In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import random
import pickle
import util
from copy import deepcopy
from genetic import Population, Chromo

img_dir = '../slides/images/'
pop_dir = '../pop_data/'
json_dir = './pop_json/'

fname = '01l_100'
img_name = img_dir + fname + '.pdf'

params = util.parse_json(json_dir + 'baseline_local.json')
print params

In [ ]:
# FIRST GENERATION OF POPULATION
pop = Population( size = 100,
        crossover = params['crossover'],
        mutation = 0.1,#mutation = params['mutation'],
        elitism = params['elitism'],
        imigration = params['imigration'],
        tour_size = params['tour_size'],
        local = params['local'])

sock = util.open_socket(5000)
Chromo.sock = sock
Chromo.cross_op = params['cross_op']

if False:
    util.send_msg(sock, 'NEWGEN\n')
    best = pop.evaluate()
    util.send_msg(sock, 'ENDGEN\n')

    # Saving the population and its fitness
    pic_name = pop_dir + fname + '.dat10'
    with open(pic_name, 'w') as pic_file:
        pickle.dump(pop.population, pic_file) # saving the population and its fitness
    original_pop = deepcopy(pop.population)
else:
    # Loading the population and its fitness
    pic_name = pop_dir + fname + '_.dat'
    pic_name = pop_dir + '01l_100_cross.dat'
    with open(pic_name, 'r') as pic_file:
        original_pop = pickle.load(pic_file)
    pop.population = deepcopy(original_pop)
    p = pop.population
        

    # PLOT
    fitness_pop = [ch[0] for ch in original_pop]

    #util.send_msg(sock, 'ENDGA\n')
    #sock.close()
    print fitness_pop, len(fitness_pop)

In [ ]:
#with open(pop_dir+'01r_100_cr_mut.dat') as pic_file:
#    pop.population = pickle.load(pic_file)

# SELECTION OF PARENTS
par_list = pop.selection()

# Flatten population
p = [pop.population[0]] # elitism
for tup in par_list:
    p.append(tup[0])
    p.append(tup[1])
p = p[:pop.size] # Truncate at population size
#fitness[-1] = pop.population[0][1] # replace the last by the best, because of elitism rate

# Saving the population and its fitness
#pic_name = pop_dir + fname + '_sel.dat'
pic_name = pop_dir + fname + '_cr_mut_sel.dat10'
with open(pic_name, 'w') as pic_file:
    pickle.dump(p, pic_file) # saving the population and its fitness

# PLOT
fitness_sel = [ch[0] for ch in p]

In [ ]:
# Crossover
child_list = []
p_list = deepcopy(par_list)
for parents in p_list:
    childs = ()

    cross = random.random() < pop.crossover
    if cross:
        print 'Y',
        childs = Chromo.crossover(parents)
    else:
        print 'N',
        childs = parents
    print
    child_list.append(childs)

# Flatten population
c = []
for tup in child_list:
    c.append(tup[0])
    c.append(tup[1])
c = c[:pop.size]

# Evaluate
pop.population = deepcopy(c)
util.send_msg(sock, 'NEWGEN\n')
best = pop.evaluate()
util.send_msg(sock, 'ENDGEN\n')

# Saving the population and its fitness
pic_name = pop_dir + fname + '_cross.dat'
pic_file = open(pic_name, 'w')
pickle.dump(pop.population, pic_file) # saving the population and its fitness
pic_file.close()

# PLOT
fitness_cross = [ch[0] for ch in pop.population]

In [ ]:
# Mutation
m = []
_p = deepcopy(p)
for ch in _p:
    mutate = random.random() < pop.mutation
    if mutate:
        print '\tY',
        ch = Chromo.mutate(ch)
    else:
        print '\tN',
    m.append(ch)
print

# Evaluate
pop.population = deepcopy(m)
util.send_msg(sock, 'NEWGEN\n')
best = pop.evaluate()
util.send_msg(sock, 'ENDGEN\n')

# Saving the population and its fitness
pic_name = pop_dir + fname + '_cr_mut.dat10'
pic_file = open(pic_name, 'w')
pickle.dump(pop.population, pic_file) # saving the population and its fitness
pic_file.close()

# PLOT
fitness_mut = [ch[0] for ch in pop.population]

In [ ]:
# Evolve as normal
pop.population = deepcopy(original_pop)
pop.evolve()

# Evaluate
util.send_msg(sock, 'NEWGEN\n')
best = pop.evaluate()
util.send_msg(sock, 'ENDGEN\n')

# Saving the population and its fitness
pic_name = pop_dir + fname + '_cr_mut.dat'
with open(pic_name, 'w') as pic_file:
    pickle.dump(pop.population, pic_file) # saving the population and its fitness

# PLOT
fitness_cr_mut = [ch[0] for ch in pop.population]

In [ ]:
#fitness = [ch[0] for ch in pop.population]
#fitness = [random.uniform(-5,5) for i in range(10**3)]
#with open(pop_dir+fname+'.dat', 'r') as infile:
#    fitness = pickle.load(infile)
plt.hist(fitness, bins='auto', normed=1)
plt.grid()
plt.title('Fitness - 1st generation with 100 chromosomes - local search')
plt.ylabel('Probability Density')
plt.xlabel('Fitness ($)')
plt.savefig(img_dir+fname+'.pdf')
#with open(pop_dir+fname+'.dat', 'w') as outfile:
#    pickle.dump(fitness, outfile)
plt.show()

In [ ]:
#np.histogram(fitness)
y, bins, patches = plt.hist(fitness, bins='auto', normed=True, facecolor='blue', alpha=0.5)
density = stats.gaussian_kde(fitness)
plt.plot(bins,density(bins),'-', color='blue')
#bincenters = 0.5*(bins[1:]+bins[:-1])
#plt.plot(bincenters,y,'--')
plt.subplots_adjust(left=0.15)
plt.show()

In [ ]:
pic_name = pop_dir + fname + '_cr_mut.dat'
pic_file = open(pic_name, 'r')
pop.population = pickle.load(pic_file) # saving the population and its fitness
pic_file.close()
# Evolve as normal
for i in range(8):
    print 'GENERATION', i+3
    pop.evolve()

    # Evaluate
    util.send_msg(sock, 'NEWGEN\n')
    best = pop.evaluate()
    util.send_msg(sock, 'ENDGEN\n')

# Saving the population and its fitness
pic_name = pop_dir + '10l_100' + '.dat'
pic_file = open(pic_name, 'w')
pickle.dump(pop.population, pic_file) # saving the population and its fitness
pic_file.close()

In [ ]:
for i in range(10):
    print 'GENERATION', i+11
    pop.evolve()

    # Evaluate
    util.send_msg(sock, 'NEWGEN\n')
    best = pop.evaluate()
    util.send_msg(sock, 'ENDGEN\n')
    # Saving the population and its fitness
    pic_name = pop_dir + '20l_100' + '.dat'
    with open(pic_name, 'w') as pic_file:
        pickle.dump(pop.population, pic_file) # saving the population and its fitness

for i in range(10):
    print 'GENERATION', i+21
    pop.evolve()

    # Evaluate
    util.send_msg(sock, 'NEWGEN\n')
    best = pop.evaluate()
    util.send_msg(sock, 'ENDGEN\n')

    # Saving the population and its fitness
    pic_name = pop_dir + '30l_100' + '.dat'
    with open(pic_name, 'w') as pic_file:
        pickle.dump(pop.population, pic_file) # saving the population and its fitness

In [ ]:
print pop.population